In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("user_reviews.csv")

In [3]:
df.head()

,movieId,rating,quote,reviewId,isVerified,isSuperReviewer,hasSpoilers,hasProfanity,score,creationDate,userDisplayName,userRealm,userId
0,13905397-4673-3e05-954d-8840645bebc2,1.0,"Did.Not.Like.\r\n\r\nThis movie was boring, ov...",NaN,False,False,False,False,1.0,2001-03-07,NaN,RT,901245019
1,13905397-4673-3e05-954d-8840645bebc2,4.5,[font=Arial]i never read the original story. ...,NaN,False,False,False,False,4.5,2002-03-08,NaN,RT,901245679
2,a2b255da-eb4d-3436-804b-fbe7b8732097,3.5,[left][b][font=Book Antiqua]xxxxx[/font][/b][/...,NaN,False,False,False,False,3.5,2002-09-03,NaN,RT,900608936
3,94a040f3-e1db-33b0-b7aa-6187e1406d21,2.0,"Art films can't center around gigantic, green ...",NaN,False,False,False,False,2.0,2003-01-21,NaN,RT,900592542
4,df35d67b-c89e-30fa-bff5-c31006cc6ae4,3.5,"dude, X-men 2 rocks! better then the first one...",NaN,False,False,False,False,3.5,2003-05-02,NaN,RT,900659519


In [4]:
df=df[['quote','rating']]

In [5]:
df.head()

,quote,rating
0,"Did.Not.Like.\r\n\r\nThis movie was boring, ov...",1.0
1,[font=Arial]i never read the original story. ...,4.5
2,[left][b][font=Book Antiqua]xxxxx[/font][/b][/...,3.5
3,"Art films can't center around gigantic, green ...",2.0
4,"dude, X-men 2 rocks! better then the first one...",3.5


### Pre processing
> rating:good(3-5)=1 bad(1-2)=0

> lemmatizing

In [6]:
df['rating'].value_counts()

rating
5.0    680511
4.0    398229
4.5    268320
3.5    261552
3.0    235557
2.5    115914
2.0     97995
0.5     62763
1.0     54332
1.5     49410
Name: count, dtype: int64

In [7]:
df['rating'].unique()

array([1. , 4.5, 3.5, 2. , 3. , 4. , 5. , 2.5, 0.5, 1.5])

In [8]:
for i in df['rating'].unique():
    if i>3:
        df['rating']=df['rating'].replace({i:1})
    else:
        df['rating']=df['rating'].replace({i:0})

In [9]:
df['rating'].value_counts()

rating
1.0    1608612
0.0     615971
Name: count, dtype: int64

In [10]:
len(df['rating'])

2224583

In [11]:
from sklearn.utils import resample
df_min=df[df['rating']==0]
df_maj=df[df['rating']==1]
df_maj_downsample=resample(df_maj,replace=False,n_samples=1231942,random_state=42)
new_df=pd.concat((df_maj_downsample,df_min))
new_df.shape


(1847913, 2)

In [21]:
new_df['quote'].iloc[10]

"A Spider-Man's fan dream came true."

In [22]:
new_df['rating'].iloc[10]

1

In [14]:
new_df['rating']=df['rating'].astype(int)

In [23]:
new_df['rating'].value_counts()

rating
1    1231942
0     615971
Name: count, dtype: int64

In [15]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re

wnl = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

corpus = []
output=[]
# Processing each row in new_df
for i in range(len(new_df)):
    if i % 10000 == 0:
        print(f"completed: {i} out of 1847913")
        percent = (i / 1847913) * 100
        print(f"Completed: {percent:.2f}%")

    # Removing special characters and lowercasing
    review = re.sub('[^a-zA-Z]', ' ', new_df['quote'].iloc[i])
    y=new_df['rating'].iloc[i]
    review = review.lower()

    # Splitting into words
    review = review.split()

    # Lemmatization and stopwords removal
    review = [wnl.lemmatize(word, pos='v') for word in review if word not in stop_words]

    # Joining the processed tokens back into a string
    review = ' '.join(review)
    corpus.append(review)
    output.append(y)

completed: 0 out of 1847913
Completed: 0.00%
completed: 10000 out of 1847913
Completed: 0.54%
completed: 20000 out of 1847913
Completed: 1.08%
completed: 30000 out of 1847913
Completed: 1.62%
completed: 40000 out of 1847913
Completed: 2.16%
completed: 50000 out of 1847913
Completed: 2.71%
completed: 60000 out of 1847913
Completed: 3.25%
completed: 70000 out of 1847913
Completed: 3.79%
completed: 80000 out of 1847913
Completed: 4.33%
completed: 90000 out of 1847913
Completed: 4.87%
completed: 100000 out of 1847913
Completed: 5.41%
completed: 110000 out of 1847913
Completed: 5.95%
completed: 120000 out of 1847913
Completed: 6.49%
completed: 130000 out of 1847913
Completed: 7.03%
completed: 140000 out of 1847913
Completed: 7.58%
completed: 150000 out of 1847913
Completed: 8.12%
completed: 160000 out of 1847913
Completed: 8.66%
completed: 170000 out of 1847913
Completed: 9.20%
completed: 180000 out of 1847913
Completed: 9.74%
completed: 190000 out of 1847913
Completed: 10.28%
completed: 20

In [16]:
import pickle
with open ('corpus.pkl','wb') as cps:
    pickle.dump(corpus,cps)

In [17]:
import pickle
with open ('y.pkl','wb') as op:
    pickle.dump(output,op)